In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

train_data=pd.read_csv('Polynomialdata_train.csv')
test_data=pd.read_csv('Polynomialdata_test.csv')
Xtrain=train_data.iloc[0:50000,1:4]
ytrain=train_data.iloc[0:50000,0:1]
Xtest=test_data.iloc[0:10000,1:4]
ytest=test_data.iloc[0:10000,0:1]
X_train=Xtrain.to_numpy()
y_train=ytrain.to_numpy()
X_test=Xtest.to_numpy()
y_test=ytest.to_numpy()

def normalize_data(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std
X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

degree = 3

X_poly = X_train.copy()

for d in range(2, degree + 1):
    X_poly = np.concatenate((X_poly, X_train**d), axis=1)

num_features = X_train.shape[1]
for i in range(num_features):
    for j in range(i + 1, num_features):
        cross_product = X_train[:, i] * X_train[:, j]
        X_poly = np.concatenate((X_poly, cross_product.reshape(-1, 1)), axis=1)

X_poly = np.concatenate((np.ones((X_train.shape[0], 1)), X_poly), axis=1)

print("X_poly:\n", X_poly.shape)


X_poly:
 (50000, 13)


In [2]:
X_poly_test=X_test.copy()
for d in range(2, degree + 1):
    X_poly_test = np.concatenate((X_poly_test, X_test**d), axis=1)
num_features = X_test.shape[1]
for i in range(num_features):
    for j in range(i + 1, num_features):
        cross_product = X_test[:, i] * X_test[:, j]
        X_poly_test = np.concatenate((X_poly_test, cross_product.reshape(-1, 1)), axis=1)
X_poly_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_poly_test), axis=1)


In [3]:
X_train_poly=X_poly
X_test_poly=X_poly_test

In [4]:
class SGDRegressor:
    def __init__(self, learning_rate=0.01, epochs=100, epsilon=np.finfo(np.float64).eps, alpha=0.01):
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.epsilon = epsilon
        self.alpha = alpha

    def fit(self, X_train, y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for epoch in tqdm(range(self.epochs)):
            for i in range(X_train.shape[0]):
                X_i = X_train[i, :].reshape(1, -1)
                y_i = y_train[i]
                
                y_hat = np.dot(X_i, self.coef_.T) + self.intercept_

                intercept_der = -2 * (y_i - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * (y_i - y_hat) * X_i + 2 * self.alpha * self.coef_
                self.coef_ = self.coef_ - (self.lr * coef_der)

                # Clip coefficients to avoid overflow
                self.coef_ = np.clip(self.coef_, -1e6, 1e6)
                self.intercept_ = np.clip(self.intercept_, -1e6, 1e6)

                # Check for overflow during training
                if np.isnan(self.coef_).any() or np.isinf(self.coef_).any() or np.isnan(self.intercept_).any() or np.isinf(self.intercept_).any():
                    print("Numerical instability detected. Reduce learning rate or check data.")
                    return

        print(self.intercept_, self.coef_)

    def predict(self, X_test):
        return np.dot(X_test, self.coef_.T) + self.intercept_

gdr = SGDRegressor(epochs=1000, learning_rate=0.01, alpha=0.01)

gdr.fit(X_train_poly, y_train)

100%|██████████| 1000/1000 [1:27:08<00:00,  5.23s/it]

[[198894.53045189]] [[ 1.93624462e+05  3.77855718e+05 -7.03998399e+05 -4.49471604e+05
   6.00586451e+05  7.57752716e+05  9.73197804e+05 -2.79164769e+05
  -5.29434313e+05 -8.59498096e+05 -3.18305208e+05  9.84440148e+05
   8.98630569e+02]]


In [5]:
y_pred2=gdr.predict(X_poly_test)
y_pred1=gdr.predict(X_poly)

In [8]:
y_pred1

array([[-3419353.87797204],
       [  -13215.11970667],
       [ 9121837.48115371],
       ...,
       [ 1845817.91440162],
       [  673300.27133147],
       [ 1107250.80554083]])

In [9]:
y_pred2

array([[3552903.22529772],
       [7160239.09133921],
       [2731153.92622126],
       ...,
       [3437954.13071451],
       [  56928.35285426],
       [ 577041.41325481]])

In [6]:
def SSM(y_train):
    return np.sum((y_train-np.mean(y_train))**2)
def SSR(y_train,y_pred):
    return np.sum((y_train-y_pred)**2)
    
SSr=SSR(y_test,y_pred2)
SSm=SSM(y_train)
r2_score=1-(SSr/SSm)
r2_score

0.9503860368295035

In [7]:
def SSM(y_train):
    return np.sum((y_train-np.mean(y_train))**2)
def SSR(y_train,y_pred):
    return np.sum((y_train-y_pred)**2)
    
SSr=SSR(y_train,y_pred1)
SSm=SSM(y_train)
r2_score=1-(SSr/SSm)
r2_score

0.030623682964636467